In [1]:
import warnings

warnings.filterwarnings("ignore")

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.utils import np_utils
import numpy as np
from keras import backend as K

K.set_image_dim_ordering("th")

Using TensorFlow backend.


In [2]:
from keras.datasets import mnist

(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [3]:
num_pixels = X_train.shape[1] * X_train.shape[2]
n_channels = 1  # All picutures are in shades of gray


def preprocess(matrix):
    return (
        matrix.reshape(
            matrix.shape[0], n_channels, matrix.shape[1], matrix.shape[2]
        ).astype("float32")
        / 255.0
    )

In [4]:
print(X_train.shape)
print(X_train.dtype)
print(np.max(X_train))

(60000, 28, 28)
uint8
255


In [5]:
X_train, X_test = preprocess(X_train), preprocess(X_test)

In [6]:
print(X_train.shape)
print(X_train.dtype)
print(np.max(X_train))

(60000, 1, 28, 28)
float32
1.0


In [7]:
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_train.shape[1]
print(y_train.shape)
print(y_train.shape[1])

(60000, 10)
10


In [8]:
def basline_model():
    model = Sequential()
    model.add(Flatten(input_shape=(1, 28, 28)))
    model.add(Dense(num_pixels, activation="relu", kernel_initializer="normal"))
    model.add(Dense(num_classes, activation="softmax", kernel_initializer="normal"))
    model.compile(
        loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
    )
    return model

In [9]:
def convolution_small():
    model = Sequential()
    model.add(
        Convolution2D(
            32, 5, 5, border_mode="valid", input_shape=(1, 28, 28), activation="relu"
        )
    )
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation="relu"))
    model.add(Dense(num_classes, activation="softmax"))
    model.compile(
        loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
    )
    return model

In [10]:
def convolution_large():
    model = Sequential()
    model.add(
        Convolution2D(
            30, 5, 5, border_mode="valid", input_shape=(1, 28, 28), activation="relu"
        )
    )
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Convolution2D(15, 3, 3, activation="relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation="relu"))
    model.add(Dense(50, activation="relu"))
    model.add(Dense(num_classes, activation="softmax"))
    model.compile(
        loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
    )
    return model

In [11]:
np.random.seed(101)
models = [
    ("Baseline", basline_model()),
    ("Small", convolution_small()),
    ("Large", convolution_large()),
]
for name, model in models:
    print("Model: %s" % name)
    model.fit(
        X_train,
        y_train,
        validation_data=(X_test, y_test),
        nb_epoch=10,
        batch_size=100,
        verbose=2,
    )
    scores = model.evaluate(X_test, y_test, verbose=0)
    print("Base error: %f" % (100 - scores[1] * 100))
    print("_" * 20)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: Baseline
Instructions for updating:
Use tf.cast instead.
Train on 60000 samples, validate on 10000 samples
Epoch 1/10
 - 8s - loss: 0.2296 - acc: 0.9337 - val_loss: 0.1124 - val_acc: 0.9682
Epoch 2/10
 - 8s - loss: 0.0904 - acc: 0.9731 - val_loss: 0.0827 - val_acc: 0.9743
Epoch 3/10
 - 8s - loss: 0.0566 - acc: 0.9828 - val_loss: 0.0642 - val_acc: 0.9786
Epoch 4/10
 - 8s - loss: 0.0378 - acc: 0.9887 - val_loss: 0.0692 - val_acc: 0.9779
Epoch 5/10
 - 8s - loss: 0.0259 - acc: 0.9924 - val_loss: 0.0722 - val_acc: 0.9782
Epoch 6/10
 - 8s - loss: 0.0208 - acc: 0.9938 - val_loss: 0.0639 - val_acc: 0.9805
Epoch 7/10
 - 8s - loss: 0.0148 - acc: 0.9958 - val_loss: 0.0661 - val_acc: 0.9801
Epoch 8/10
 - 8s - loss: 0.0112 - acc: 0.9966 - val_loss: 0.0634 - val_acc: 0.9818
Epoch 9/10
 - 8s - loss: 0.0098 - a